In [1]:
import torch
import pykeen
import pandas as pd
from pykeen import predict
from pykeen.pipeline import pipeline
from pykeen.hpo import hpo_pipeline
from pykeen.triples import TriplesFactory
import numpy as np
from pykeen.utils import set_random_seed


file_path = 'C.1_query.tsv'
tf = TriplesFactory.from_path(file_path, delimiter="\t")
training, testing = tf.split([0.85, 0.15], random_state=2025)
training, validation = training.split([0.8, 0.2], random_state=2025)
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
print(f"Total triples: {len(tf.mapped_triples)}")
print(f"Training triples: {len(training.mapped_triples)}")
print(f"Validation triples: {len(validation.mapped_triples)}")
print(f"Testing triples: {len(testing.mapped_triples)}")

Total triples: 1287
Training triples: 874
Validation triples: 219
Testing triples: 194


# Model Hyperparameter Tuning

In [3]:
def run_pipeline(model_name, model_kwargs_range, study_name, random_seed=2025):
    # Set the random seed for reproducibility
    set_random_seed(random_seed)

    # Validate input
    if model_name is None or model_kwargs_range is None:
        raise ValueError("Both model name and model parameter ranges must be provided.")

    # Run the hyperparameter optimization pipeline
    result = hpo_pipeline(
        training=training,
        testing=testing,
        validation=validation,

        # Hyperparameter optimization configuration
        sampler="tpe",
        n_trials=30,

        study_name=study_name,

        # Training parameter ranges
        training_kwargs_ranges=dict(
            num_epochs=dict(type="int", low=50, high=150, step=50)
        ),

        # Model and its parameter search space
        model=model_name,
        model_kwargs=dict(
            random_seed=random_seed
        ),
        model_kwargs_ranges=model_kwargs_range,

        # Optimizer configuration
        optimizer="adam",
        optimizer_kwargs_ranges=dict(
            lr=dict(type="float", low=0.0001, high=0.001, log=True),
            weight_decay=dict(type="float", low=1e-4, high=1e-3, log=True)
        ),

        # Negative sampling configuration
        negative_sampler="basic",
        negative_sampler_kwargs_ranges=dict(
            num_negs_per_pos=dict(type="int", low=1, high=10, step=3)
        ),

        # Early stopping settings
        stopper="early",
        stopper_kwargs=dict(
            patience=10,
            frequency=5,
            metric="hits@5",
            relative_delta=0.002,
            larger_is_better=True,
        ),

        device=device,
    )

    return result

# TransH Model

In [4]:
model_name = "TransH"
model_kwargs_range =dict(
            embedding_dim=dict(type="int", low=128, high=256, step=64)
        )

# Run the HPO pipeline
resultTransH = run_pipeline(model_name, model_kwargs_range, 'TransH_HPO_Experiment')

[I 2025-06-11 00:28:30,960] A new study created in memory with name: TransH_HPO_Experiment
No random seed is specified. Setting to 891829896.


Training epochs on cuda:0:   0%|          | 0/150 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0.00/1.00 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/1.00 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/1.00 [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0.00/1.00 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/1.00 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/1.00 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/1.00 [00:00<?, ?batch/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 0.08s seconds
INFO:pykeen.hpo.hpo:Pruned trial: <optuna.trial._trial.Trial object at 0x000001EBE094AED0> at epoch 5 due to both.realistic.inverse_harmonic_mean_rank=0.01141552511415525
[I 2025-06-11 00:36:00,394] Trial 29 pruned. 


In [5]:
resultTransH.save_to_directory('transh_hpo_result')

# RotatE Model

In [6]:
model_name = "RotatE"
model_kwargs_range =dict(
            embedding_dim=dict(type="int", low=128, high=256, step=64),
        )
# Run the HPO pipeline
resultRotatE = run_pipeline(model_name, model_kwargs_range, 'RotatE_HPO_Experiment')

[I 2025-06-11 00:36:00,483] A new study created in memory with name: RotatE_HPO_Experiment
INFO:pykeen.hpo.hpo:Using model: <class 'pykeen.models.unimodal.rotate.RotatE'>
INFO:pykeen.hpo.hpo:Using loss: <class 'pykeen.losses.MarginRankingLoss'>
INFO:pykeen.hpo.hpo:Using optimizer: <class 'torch.optim.adam.Adam'>
INFO:pykeen.hpo.hpo:Using training loop: <class 'pykeen.training.slcwa.SLCWATrainingLoop'>
INFO:pykeen.hpo.hpo:Using negative sampler: <class 'pykeen.sampling.basic_negative_sampler.BasicNegativeSampler'>
INFO:pykeen.hpo.hpo:Using evaluator: <class 'pykeen.evaluation.rank_based_evaluator.RankBasedEvaluator'>
INFO:pykeen.hpo.hpo:Attempting to maximize both.realistic.inverse_harmonic_mean_rank
INFO:pykeen.hpo.hpo:Filter validation triples when testing: True
INFO:pykeen.pipeline.api:Using device: cuda
INFO:pykeen.nn.representation:Inferred unique=False for Embedding()
INFO:pykeen.nn.representation:Inferred unique=False for Embedding()
INFO:pykeen.stoppers.early_stopping:Inferred c

Training epochs on cuda:0:   0%|          | 0/50 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0.00/4.00 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/4.00 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/4.00 [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0.00/4.00 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/4.00 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/4.00 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/4.00 [00:00<?, ?batch/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 0.08s seconds
INFO:pykeen.hpo.hpo:Pruned trial: <optuna.trial._trial.Trial object at 0x000001EBE7BFC450> at epoch 5 due to both.realistic.inverse_harmonic_mean_rank=0.045662100456621
[I 2025-06-11 00:46:23,934] Trial 29 pruned. 


In [7]:
resultRotatE.save_to_directory('rotate_hpo_result')

# ComplEx Model

In [8]:
model_name = "ComplEx"
model_kwargs_range =dict(
            embedding_dim=dict(type="int", low=128, high=256, step=64),
        )
# Run the HPO pipeline
resultComplEx = run_pipeline(model_name, model_kwargs_range, 'ComplEx_HPO_Experiment')

[I 2025-06-11 00:46:24,000] A new study created in memory with name: ComplEx_HPO_Experiment
INFO:pykeen.hpo.hpo:Using model: <class 'pykeen.models.unimodal.complex.ComplEx'>
INFO:pykeen.hpo.hpo:Using loss: <class 'pykeen.losses.SoftplusLoss'>
INFO:pykeen.hpo.hpo:Using optimizer: <class 'torch.optim.adam.Adam'>
INFO:pykeen.hpo.hpo:Using training loop: <class 'pykeen.training.slcwa.SLCWATrainingLoop'>
INFO:pykeen.hpo.hpo:Using negative sampler: <class 'pykeen.sampling.basic_negative_sampler.BasicNegativeSampler'>
INFO:pykeen.hpo.hpo:Using evaluator: <class 'pykeen.evaluation.rank_based_evaluator.RankBasedEvaluator'>
INFO:pykeen.hpo.hpo:Attempting to maximize both.realistic.inverse_harmonic_mean_rank
INFO:pykeen.hpo.hpo:Filter validation triples when testing: True
INFO:pykeen.pipeline.api:Using device: cuda
INFO:pykeen.nn.representation:Inferred unique=False for Embedding(
  (regularizer): LpRegularizer()
)
INFO:pykeen.nn.representation:Inferred unique=False for Embedding(
  (regularizer)

Training epochs on cuda:0:   0%|          | 0/150 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0.00/7.00 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/7.00 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/7.00 [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0.00/55.0 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/55.0 [00:00<?, ?batch/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 0.08s seconds


Evaluating on cuda:0:   0%|          | 0.00/219 [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 0.07s seconds
[I 2025-06-11 00:54:39,763] Trial 29 finished with value: 0.023986712098121643 and parameters: {'model.embedding_dim': 192, 'optimizer.lr': 0.0007252267916688366, 'optimizer.weight_decay': 0.0005851059396488976, 'negative_sampler.num_negs_per_pos': 1, 'training.num_epochs': 50, 'training.batch_size': 16}. Best is trial 29 with value: 0.023986712098121643.


In [9]:
resultComplEx.save_to_directory('complex_hpo_result')

# ConvKB Model

In [10]:
model_name = "ConvKB"
model_kwargs_range =dict(
            embedding_dim=dict(type="int", low=128, high=256, step=64),
            hidden_dropout_rate=dict(type="float", low=0.3, high=0.5, step=0.1),
            num_filters=dict(type="int", low=64, high=128, step=32),
        )
# Run the HPO pipeline
resultConvKB = run_pipeline(model_name, model_kwargs_range, 'ConvKB_HPO_Experiment')

[I 2025-06-11 00:54:39,828] A new study created in memory with name: ConvKB_HPO_Experiment
INFO:pykeen.hpo.hpo:Using model: <class 'pykeen.models.unimodal.conv_kb.ConvKB'>
INFO:pykeen.hpo.hpo:Using loss: <class 'pykeen.losses.MarginRankingLoss'>
INFO:pykeen.hpo.hpo:Using regularizer: <class 'pykeen.regularizers.LpRegularizer'>
INFO:pykeen.hpo.hpo:Using optimizer: <class 'torch.optim.adam.Adam'>
INFO:pykeen.hpo.hpo:Using training loop: <class 'pykeen.training.slcwa.SLCWATrainingLoop'>
INFO:pykeen.hpo.hpo:Using negative sampler: <class 'pykeen.sampling.basic_negative_sampler.BasicNegativeSampler'>
INFO:pykeen.hpo.hpo:Using evaluator: <class 'pykeen.evaluation.rank_based_evaluator.RankBasedEvaluator'>
INFO:pykeen.hpo.hpo:Attempting to maximize both.realistic.inverse_harmonic_mean_rank
INFO:pykeen.hpo.hpo:Filter validation triples when testing: True
INFO:pykeen.pipeline.api:Using device: cuda
INFO:pykeen.nn.representation:Inferred unique=False for Embedding()
INFO:pykeen.nn.representation:

Training epochs on cuda:0:   0%|          | 0/150 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0.00/2.00 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/2.00 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/2.00 [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0.00/2.00 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/2.00 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/2.00 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/2.00 [00:00<?, ?batch/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 0.87s seconds
INFO:pykeen.hpo.hpo:Pruned trial: <optuna.trial._trial.Trial object at 0x000001EC0A390350> at epoch 5 due to both.realistic.inverse_harmonic_mean_rank=0.09360730593607305
[I 2025-06-11 01:17:23,031] Trial 29 pruned. 


In [11]:
resultConvKB.save_to_directory('convkb_hpo_result')